In [11]:
import os
import pandas as pd

import torch
import torchaudio
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import openhd as hd
from torch import nn

In [12]:
!rm -rf ~/.openhd

In [13]:
class UrbanSoundDataset(Dataset):

    def __init__(self, annotations_file, audio_dir,transformation,target_sample_rate,num_samples,device):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.device = device
        self.transformation = transformation.to(self.device)
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)
        # signal -> (num_channels,samples) -> (2,16000) -> (1,16000)
        signal = self._resample_if_necessary(signal,torch.scalar_tensor(sr).cuda())
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        signal = signal.reshape((signal.shape[1]* signal.shape[2]))
        return signal, label
    
    def _cut_if_necessary(self, signal):
        # signal -> Tensor -> (1, num_samples) -> (1,50000) -> (1,22050)
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        return signal
    
    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            # [1,1,1] -> [1,1,1,0,0]
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal
    
    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1: # (2,16000)
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal


    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir, fold, self.annotations.iloc[
            index, 0])
        return path

    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]


In [14]:
ANNOTATIONS_FILE = './UrbanSound8K/metadata/UrbanSound8K.csv'
AUDIO_DIR = './UrbanSound8K/audio'
SAMPLE_RATE = 22050
NUM_SAMPLES = 22050

In [15]:
# if torch.cuda.is_available():
#     device = 'cuda'
# else:
#     device = 'cpu'
# print(f"Using device {device}")
    
device = 'cpu'

In [16]:
mel_spectogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024, #frame size
    hop_length=512,
    n_mels=64
)

In [17]:
usd = UrbanSoundDataset(ANNOTATIONS_FILE, AUDIO_DIR,mel_spectogram,SAMPLE_RATE,NUM_SAMPLES,'cpu')
# print(f"There are {len(usd)} samples in the dataset.")
# signal, label = usd[1]

In [18]:
def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader

In [43]:
train_dataloader = create_data_loader(usd, 128)

In [20]:
feature_matrix, labels= next(iter(train_dataloader))

feature_matrix = np.array(feature_matrix, dtype = np.float32)

feature_matrix = hd.utils.MatrixNormalizer().norm(feature_matrix)

In [23]:
feature_matrix

array([[4.9270888e-08, 3.5907703e-09, 1.8523517e-08, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.3964677e-06, 1.0328251e-05, 2.0932032e-06, ..., 4.5639084e-07,
        3.8011061e-07, 6.2770800e-07],
       [1.6010954e-06, 1.2469583e-06, 1.5438469e-05, ..., 6.5716314e-08,
        1.8070310e-07, 5.2935985e-07],
       ...,
       [3.8166851e-05, 2.5097384e-06, 1.4485910e-05, ..., 3.0962062e-07,
        1.2716031e-07, 1.3239450e-07],
       [1.8354552e-04, 8.2021143e-05, 9.3732097e-06, ..., 3.3753000e-07,
        1.9104503e-07, 2.4021546e-07],
       [9.4476800e-06, 4.9112616e-05, 2.6623686e-05, ..., 4.2322233e-07,
        2.4372088e-07, 1.3396875e-07]], dtype=float32)

In [24]:
N = feature_matrix.shape[0]
F = feature_matrix.shape[1]
N, F

(4096, 2816)

In [25]:
labels = np.array(labels, dtype = np.int32)

In [26]:
n_classes = 10

In [27]:
Q = 10
D = 10000
hd.init(D=D, context=globals())


In [28]:
@hd.run
def create_random_bases():
    id_base = hd.draw_random_hypervector()
    level_base = hd.draw_random_hypervector()
    return id_base, level_base


@hd.run
def create_ids(F, id_base):
    id_hvs = hd.hypermatrix(F) # np.zeros(F, N) (not the empty list) 
    for f in range(F):
        id_hvs[f] = hd.permute(id_base, f)

    return id_hvs

@hd.run
def create_levels(Q, level_base):
    level_hvs = hd.hypermatrix(Q+1) # np.zeros((Q+1), N) (not the empty list)
    for q in range(Q+1):
        idx = int(q/float(Q) * D) / 2
        level_hvs[q] = hd.flip(level_base, idx)
        level_hvs[q] = hd.shuffle(level_hvs[q], 0)

    return level_hvs

In [29]:
with hd.utils.timing("Base hypervectors"):
    id_base, level_base = create_random_bases()
    id_hvs = create_ids(F, id_base)
    level_hvs = create_levels(Q, level_base)


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [__ARG__id_base : __one__]
  <class '_ast.Assign'> :  [__ARG__level_base : __one__]

{'id_base': 'hypervec_type', 'level_base': 'hypervec_type', '__ARG__id_base': 'float*', '__ARG__level_base': 'float*', 'id_base_22b19319': <class 'float'>, 'level_base_5541502b': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [__ARG__id_hvs : __ARG__id_base]

{'f': <class 'int'>, 'F': <class 'int'>, 'id_hvs': 'hypermat_type', 'id_base': 'hypervec_type', '__ARG__id_base': 'float*', '__ARG__id_hvs': 'float*', 'id_base_f_16a56737': <class 'float'>, 'id_hvs_f_00177a23': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [level_hvs_q_7acb546c : __ARG__level_base]
  <class '_ast.Assign'> :  [__ARG__level_hvs : level_hvs_q_7acb546c]

{'idx': <class 'int'>, 'q': 'int', 'Q': <class 'int'>, 'D': 

In [30]:
def preprocesser(
        org_feature, cnv_feature, # Predefined argument (single feature)
        Q, level_hvs, id_hvs): # arguments passed by args
    cnv_feature = int(org_feature * Q)


def encoder(
        input_features, output_hypervector, # Predefined arguments
        Q, level_hvs, id_hvs): # arguments passed by args
    for f in range(F):
        output_hypervector += level_hvs[input_features[f]] * id_hvs[f]

In [31]:
with hd.utils.timing("Encode training"):
    hv_matrix = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = feature_matrix,
            preprocess_function = preprocesser # optional
            )

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'org_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'cnv_feature': <class 'int'>, 'Q': <class 'int'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int

In [32]:
@hd.run
def single_pass(hv_matrix, labels, N, n_classes):
    class_hvs = hd.hypermatrix(n_classes)

    for idx in range(N):
        class_hvs[labels[idx]] += hv_matrix[idx]

    return class_hvs

In [40]:
with hd.utils.timing("Single pass"):
    class_hvs = single_pass(hv_matrix, labels, N, n_classes)
    class_hvs.to_numpy()

Single pass	0.011505365371704102


In [34]:
def validate(labels, pred_labels):
    n_correct = (pred_labels == labels).sum()
    n_labels = len(labels)
    print(n_correct, n_labels, n_correct / float(n_labels) * 100)


# Guided Training

In [35]:
@hd.run
def retrain(class_hvs, hv_matrix, labels, N, n_classes):
    search_results = hd.search(class_hvs, hv_matrix)

    for idx in range(N):
        if search_results[idx] != labels[idx]:
            class_hvs[labels[idx]] += hv_matrix[idx]
            class_hvs[search_results[idx]] -= hv_matrix[idx]

    return class_hvs


In [36]:
RETRAIN_ITERATIONS = 100
SHOW_STEP_RESULT = True
for it in range(RETRAIN_ITERATIONS):
    with hd.utils.timing("Retrain itereation: %d" % it):
        class_hvs = retrain(class_hvs, hv_matrix, labels, N, n_classes)

    if SHOW_STEP_RESULT and labels is not None:
        validate(labels, hd.search(class_hvs, hv_matrix).to_numpy())

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [hv_matrix_idx_ead026f6 : __ARG__hv_matrix]
    <class '_ast.If'> : 
      then : 
        <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + hv_matrix_idx_ead026f6]
        <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + hv_matrix_idx_ead026f6]
      else : 

{'idx': <class 'int'>, 'N': <class 'int'>, 'search_results': 'np_int_array_type', 'labels': 'np_int_array_type', 'class_hvs': 'hypermat_type', 'hv_matrix': 'hypermat_type', '__ARG__class_hvs': 'float*', '__ARG__hv_matrix': 'float*', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', '__ARG__search_results': 'int*', '__ARG__search_results__STRIDE__': 'const int', 'search_results_idx_b85d35f6': <class 'int'>, 'labels_idx_b9c77ce4': <class 'int'>, 'hv_matrix_idx_ead026f6': <class 'float'>, 'class_hvs_idx_labels_df62a0b1': <class 'float'>, 'class_hvs_idx_search_results_572d84

In [41]:
@hd.run
def assoc_search(class_hvs, hv_matrix_tst):
    ret = hd.search(class_hvs, hv_matrix_tst)
    return ret

In [42]:
with hd.utils.timing("Testing with class model\n"):
    search_results = assoc_search(class_hvs, hv_matrix)

# search_results.to_numpy()

validate(labels, search_results.to_numpy())


Testing with class model
	0.004705667495727539
1324 4096 32.32421875


# Epoch based training

In [45]:
@hd.run
def epoch_train(class_hvs, hv_matrix, labels, N):
    for idx in range(N): # Iterate through each image
        class_hvs[labels[idx]] += hv_matrix[idx]
    return class_hvs

In [49]:
BATCH_SIZE = 256
from torch.utils.data import DataLoader
train_dataloader = create_data_loader(usd, BATCH_SIZE)

In [50]:
EPOCHS = 10

class_hvs = hd.hypermatrix(n_classes)
train_performances = []
for e in range(EPOCHS):
    # Fetch Current Feature batch
    feature_matrix, labels= next(iter(train_dataloader))

    feature_matrix = np.array(feature_matrix, dtype = np.float32)

    feature_matrix = hd.utils.MatrixNormalizer().norm(feature_matrix)
    
    labels = np.array(labels, dtype = np.int32)
    # Encode the signals of this batch
    hv_matrix = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = feature_matrix,
            preprocess_function = preprocesser # optional
            )

    # add to class_hvs
    class_hvs = epoch_train(class_hvs, hv_matrix, labels, BATCH_SIZE)

    v = validate(labels, hd.search(class_hvs, hv_matrix).to_numpy())
    train_performances.append(v)
    print("At epoch ",e, ": ", v)

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'org_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'cnv_feature': <class 'int'>, 'Q': <class 'int'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int

In [51]:
with hd.utils.timing("Testing with class model\n"):
    search_results = assoc_search(class_hvs, hv_matrix)

# search_results.to_numpy()

validate(labels, search_results.to_numpy())

Testing with class model
	0.01778268814086914
194 256 75.78125
